In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [117]:
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import oandapyV20.endpoints.forexlabs as labs
import oandapyV20.endpoints.orders as ordersEndpoint
import oandapyV20.endpoints.trades as tradesEndpoint

### Order book data scrapped using different notebook

In [3]:
orders = pd.concat([
    pd.read_csv('order_book/january.csv'),
    pd.read_csv('order_book/january_missing.csv'),
    pd.read_csv('order_book/february.csv'),
    pd.read_csv('order_book/march.csv'),
])

In [4]:
positions = pd.concat([
    pd.read_csv('position_book/january.csv'),
    pd.read_csv('position_book/february.csv'),
    pd.read_csv('position_book/march.csv'),
])

### Functions for getting candle data

In [5]:
def hist(api, instrument, start_days, end_days, granularity):

    start_date = (dt.datetime.now()-dt.timedelta(days=start_days)).strftime('%Y-%m-%dT%H:%M:%SZ')
    end_date = (dt.datetime.now()-dt.timedelta(days=end_days, hours=2,minutes=4)).strftime('%Y-%m-%dT%H:%M:%SZ')

    params ={
                "from": start_date,
                "to": end_date,
                "granularity":granularity,
            }

    df_list = []
    for r in InstrumentsCandlesFactory(instrument=instrument,params=params):
        api.request(r)
        df = pd.DataFrame(r.response['candles'])
        if(df.empty==False):
            time = df['time']
            volume = pd.DataFrame(df['volume'].apply(pd.Series))
            df = pd.DataFrame(df['mid'].apply(pd.Series))
            df = pd.concat([df,time,volume], axis=1)
            df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%dT%H:%M:%S.000000000Z')
            #df.set_index('time',inplace=True)
            df_list.append(df)
    
    final = pd.concat(df_list)
    
    names = {
        'o': 'open',
        'c': 'close',
        'h': 'high',
        'l': 'low',
        0: 'vol',
        'time': 'time',
    }
    new_names = []
    for column_name in final.columns:
        new_names.append(names[column_name])
    final.columns = new_names
    
    return final

In [6]:
def cal(client, instrument, perdiod):

    
    
    params = {
        "instrument": instrument,
        "period": perdiod
    }
    
    # PERIOD VALUES
    #3600 - 1 hour
    #43200 - 12 hours
    #86400 - 1 day
    #604800 - 1 week
    #2592000 - 1 month
    #7776000 - 3 months
    #15552000 - 6 months
    #31536000 - 1 year
    # http://developer.oanda.com/rest-live/forex-labs/

    r = labs.Calendar(params=params)
    client.request(r)
    
    df = pd.DataFrame.from_dict(r.response, orient='columns')
    
    df['timestamp'] = pd.to_datetime(df['timestamp']*1000000000)
    df = df[['impact', 'timestamp']]
    df.columns = ['impact', 'time']

    return df.groupby('time').sum().reset_index()

In [7]:
def merge(history, calendar):
    return pd.merge(history, calendar, left_on = 'time', right_on = 'time', how='outer')\
                                                                            .set_index('time')\
                                                                            .astype(float)\
                                                                            .fillna(0)

In [8]:
def broaden_impact(df, period):
    df = df.reset_index().sort_values('time').set_index('time')

    for i in range(periods):
        df.loc[
            (df['impact'].shift(-1-i) != 0)
        ,'impact'] = df['impact'].shift(-1-i)
    
    df['impact'] = df['impact'].fillna(0)

    return df[df['low'] != 0]

## Get needed data

In [157]:
client = API(access_token='f8599fa0624567b98d6293acc87489bb-e288ec05b46b6e3d0bc753e6a2fbab48')

In [158]:
granularity_param = 'M5'

In [159]:
calendar = cal(client, 'GBP_USD', 7776000)

In [160]:
history = hist(client, 'GBP_USD', 90, 0, granularity_param)

In [161]:
merged = merge(history, calendar)

In [162]:
merged.head(1)

,open,high,low,close,vol,impact
time,,,,,,
2019-01-09 21:55:00,1.27884,1.27918,1.27857,1.27908,45.0,0.0


In [163]:
orders.head(1)

,Unnamed: 0_orders,time_orders,price_orders,roundedPrice_orders,level_0_l_orders,level_0_s_orders,level_1_up_l_orders,level_1_up_s_orders,level_1_down_l_orders,level_1_down_s_orders,...,level_3_down_l_orders,level_3_down_s_orders,level_4_up_l_orders,level_4_up_s_orders,level_4_down_l_orders,level_4_down_s_orders,level_5_up_l_orders,level_5_up_s_orders,level_5_down_l_orders,level_5_down_s_orders
time,,,,,,,,,,,,,,,,,,,,,
2019-01-03 15:40:00,0,2019-01-03T15:40:00Z,1.13998,1.14,0.0065,0.0978,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0


In [164]:
positions.head(1)

,Unnamed: 0_positions,time_positions,price_positions,roundedPrice_positions,level_0_l_positions,level_0_s_positions,level_1_up_l_positions,level_1_up_s_positions,level_1_down_l_positions,level_1_down_s_positions,...,level_3_down_l_positions,level_3_down_s_positions,level_4_up_l_positions,level_4_up_s_positions,level_4_down_l_positions,level_4_down_s_positions,level_5_up_l_positions,level_5_up_s_positions,level_5_down_l_positions,level_5_down_s_positions
time,,,,,,,,,,,,,,,,,,,,,
2019-01-01,0,2019-01-01T00:00:00Z,1.14649,1.1465,0.3569,0.1814,0.272,0.0848,0.272,0.0848,...,0.1463,0.0936,0.0,0.0,0.0,0.0,0,0,0,0


In [165]:
orders = orders.set_index(pd.to_datetime(orders['time'], format="%Y-%m-%dT%H:%M:%SZ"))

KeyError: 'time'

### A bit of cleaning

In [ ]:
ordersNewColumns = []
for column in orders.columns:
    ordersNewColumns.append(column+'_orders')

In [ ]:
orders.columns = ordersNewColumns

In [ ]:
positions = positions.set_index(pd.to_datetime(positions['time'], format="%Y-%m-%dT%H:%M:%SZ"))

In [ ]:
positionsNewColumns = []
for column in positions.columns:
    positionsNewColumns.append(column+'_positions')

In [ ]:
positions.columns = positionsNewColumns

In [169]:
#base = merged.join(orders).join(positions)\
#    .dropna()\
#    .drop(['Unnamed: 0_orders', 'Unnamed: 0_positions', 'price_orders',
#          'roundedPrice_orders', 'price_positions', 'roundedPrice_positions',
#          'time_orders', 'time_positions'], axis = 1)\
base = merged[merged.index.minute.isin([0,20,40])]

In [170]:
base.head(1)

,open,high,low,close,vol,impact
time,,,,,,
2019-01-09 22:00:00,1.27908,1.27927,1.27898,1.27926,8.0,0.0


In [171]:
len(base)/72 # more less number of days

63.55555555555556

In [172]:
base['ma5'] = base['close'].rolling(5).mean()
base['ma10'] = base['close'].rolling(10).mean()
base = base.dropna()

/home/jacek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/jacek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Prepare additional parameters

In [174]:
averageMultiplayer = 1.3
bottomBarrierPips = 0.0040
endingHour = 12
openingInterval = 1
slPips = 0.0015
startingHour = 8
tpMultiplier = 1.4

tradeUnits = 100
instrument = 'EUR_USD'

if instrument == 'EUR_USD':
    spreadPips = 0.0001
elif instrument == 'GBP_USD':
    spreadPips = 0.0002
else:
    spreadPips = 0.0003

## Test many parameters for this strategy

In [181]:
df_list = []

for date in sorted(set(base.index.date)):
    for startingHour in [14,15]:
        for openingInterval in [1]:
            for bottomBarrierPips in [0.0060, 0.0050, 0.0040]:
                # TO BE DONE - prepare data with order book
                
                openTradesDF = base[
                    (
                        base.index.date == date
                    ) &
                    (
                        (
                            (base.index.hour >= startingHour) &
                            (base.index.hour < startingHour + openingInterval) &
                            (base.index.minute != 0)
                        ) |
                        (
                            (base.index.hour == startingHour + openingInterval) &
                            (base.index.minute == 0)
                        )
                    )
                ]
                
                firstCandle = base[
                    (base.index.date == date) &
                    (base.index.hour >= startingHour) &
                    (base.index.hour < startingHour + openingInterval) &
                    (base.index.minute == 0)
                ]
                
                if len(openTradesDF) == 3 * openingInterval and\
                    len(openTradesDF[openTradesDF['impact'] == 1]) == 0:
                    openTradePrices = []
                    basePrice = firstCandle.iloc[0]['open']
                    bottomBarrier = basePrice - bottomBarrierPips
                    
                    for i in range(len(openTradesDF)):
                        price = openTradesDF.iloc[i]['open']
                        
                        if openTradePrices == []:
                            if\
                                openTradesDF.iloc[i]['ma5'] < openTradesDF.iloc[i]['ma10'] and\
                                price > bottomBarrier and\
                                price < basePrice:
                                
                                openTradePrices.append(price)
                        else:
                            if\
                                openTradesDF.iloc[i]['ma5'] < openTradesDF.iloc[i]['ma10'] and\
                                price > bottomBarrier and\
                                price < basePrice and\
                                price < min(openTradePrices):
                                
                                openTradePrices.append(price)
                                
                    if openTradePrices != []:
                        for averageMultiplayer in [1.20, 1.30, 1.4, 1.5]: #
                            
                            weightedPrices = []
                            weights = []
                            for i, price in enumerate(openTradePrices):
                                if i == 0:
                                    weightedPrices.append(price)
                                    weights.append(1)
                                else:
                                    weightedPrices.append(price * averageMultiplayer * i)
                                    weights.append(averageMultiplayer * i)
                            averageOpenPrice = sum(weightedPrices) / sum(weights)
                            numberOfTrades = len(openTradePrices)
                            
                        for slPips in [0.0010, 0.0015, 0.0020]:
                            for tpMultiplier in [1.3, 1.4, 1.5,1.6]:
                                for endingHour in [18,19,20]:
                                    for moveSlPips in [0.0000, 0.0005, 0.0010, 0.0015]:
                                    
                                        checkTradesDF = base[
                                            (base.index.date == date) &
                                            (base.index.hour >= startingHour + openingInterval) &
                                            (base.index.hour <= endingHour)
                                        ].iloc[:-2]

                                        if len(checkTradesDF[checkTradesDF['impact'] == 1]) == 0:

                                            # edge case: one trade which is very low is under sl lvl
                                            slPrice = averageOpenPrice - slPips
                                            tpPrice = averageOpenPrice + (slPips * tpMultiplier)
                                            resultPips = 0.0

                                            if checkTradesDF.iloc[0]['open'] < bottomBarrier:
                                                resultPips = (bottomBarrier - averageOpenPrice)\
                                                                * numberOfTrades
                                            if checkTradesDF.iloc[0]['open'] < slPrice:
                                                resultPips = (slPrice - averageOpenPrice)\
                                                                * numberOfTrades
                                            else:
                                                closeTradeFlag = False
                                                middleHour = round((endingHour - startingHour) / 2, 0)
                                                
                                                for candle in checkTradesDF\
                                                    .iloc[1:][['high', 'low', 'open']]\
                                                    .reset_index()\
                                                    .to_dict('rows'):

                                                    if candle['time'].hour == middleHour and\
                                                        candle['time'].minute == 20:
                                                        if updatedSlPips < slPips:
                                                            testNewSlPrice = slPrice + moveSlPips
                                                            if testNewSlPrice + 0.0005 < candle['open']:
                                                                slPrice = testNewSlPrice

                                                    if candle['low'] < slPrice:
                                                        resultPips = (slPrice - averageOpenPrice)\
                                                                * numberOfTrades
                                                        closeTradeFlag = True
                                                        break
                                                    if candle['high'] > tpPrice:
                                                        resultPips = (tpPrice - averageOpenPrice)\
                                                                * numberOfTrades
                                                        closeTradeFlag = True
                                                        break

                                            if closeTradeFlag == False:
                                                resultPips = (checkTradesDF.iloc[-1]['open'] - averageOpenPrice - spreadPips)\
                                                                * numberOfTrades



                                            row = {
                                                'date': date,
                                                'startingHour': startingHour,
                                                'openingInterval': openingInterval,
                                                'bottomBarrierPips': bottomBarrierPips,
                                                'averageMultiplayer': averageMultiplayer,
                                                'slPips': slPips,
                                                'tpMultiplier': tpMultiplier,
                                                'endingHour': endingHour,
                                                'resultPips': resultPips,
                                                'moveSlPips': moveSlPips,
                                            }

                                            df_list.append(row)

In [182]:
scores = pd.DataFrame(df_list)

In [183]:
grouped = scores.groupby([
    'averageMultiplayer',
    'bottomBarrierPips',
    'endingHour',
    'openingInterval',
    'slPips',
    'startingHour',
    'tpMultiplier',
    'moveSlPips',
]).agg({'resultPips': 'sum', 'date': 'count'})\
    .sort_values([
    'resultPips', 
    'moveSlPips',
    'bottomBarrierPips'
], ascending=[
    False,
    False,
    True
])\

### Show the best strategies

In [184]:
grouped.head(10)

resultPips  \
averageMultiplayer bottomBarrierPips endingHour openingInterval slPips startingHour tpMultiplier moveSlPips               
1.5                0.004             19         1               0.002  14           1.6          0.0015         0.03437   
                                                                                                 0.0010         0.03437   
                                                                                                 0.0005         0.03437   
                                                                                                 0.0000         0.03437   
                   0.005             19         1               0.002  14           1.6          0.0015         0.03237   
                   0.006             19         1               0.002  14           1.6          0.0015         0.03237   
                   0.005             19         1               0.002  14           1.6          0.0010         0.03237   
                   0.006             19         1               0.002  14           1.6          0.0010         0.03237   
                   0.005             19         1               0.002  14           1.6          0.0005         0.03237   
                   0.006             19         1               0.002  14           1.6          0.0005         0.03237   

                                                                                                             date  
averageMultiplayer bottomBarrierPips endingHour openingInterval slPips startingHour tpMultiplier moveSlPips        
1.5                0.004             19         1               0.002  14           1.6          0.0015        33  
                                                                                                 0.0010        33  
                                                                                                 0.0005        33  
                                                                                                 0.0000        33  
                   0.005             19         1               0.002  14           1.6          0.0015        33  
                   0.006             19         1               0.002  14           1.6          0.0015        33  
                   0.005             19         1               0.002  14           1.6          0.0010        33  
                   0.006             19         1               0.002  14           1.6          0.0010        33  
                   0.005             19         1               0.002  14           1.6          0.0005        33  
                   0.006             19         1               0.002  14           1.6          0.0005        33

### Calculate percent of good strategies

In [185]:
# percentage of 'good' strategies
len(grouped[grouped['resultPips'] > 0])/len(grouped)*100

76.85185185185185

In [186]:
grouped.to_csv('Simple_afternoon_eur_usd.csv')

## Take a closer look at order book data

In [34]:
grouped.reset_index().drop(['resultPips', 'date'], axis=1).columns

Index(['averageMultiplayer', 'bottomBarrierPips', 'endingHour',
       'openingInterval', 'slPips', 'startingHour', 'tpMultiplier',
       'moveSlPips'],
      dtype='object')

In [35]:
scores.head(1)

,averageMultiplayer,bottomBarrierPips,date,endingHour,moveSlPips,openingInterval,resultPips,slPips,startingHour,tpMultiplier
0,1.5,0.006,2019-01-10,11,0.0,1,-0.003,0.001,6,1.2


In [36]:
grouped.iloc[0].name

(1.5, 0.006, 12, 1, 0.0015, 8, 1.4, 0.001)

In [37]:
columns = grouped.reset_index().drop(['resultPips', 'date'], axis=1).columns
for i in range(len(columns)):
    print('(scores["'+columns[i]+'"] == '+str(grouped.iloc[0].name[i])+") &")

(scores["averageMultiplayer"] == 1.5) &
(scores["bottomBarrierPips"] == 0.006) &
(scores["endingHour"] == 12) &
(scores["openingInterval"] == 1) &
(scores["slPips"] == 0.0015) &
(scores["startingHour"] == 8) &
(scores["tpMultiplier"] == 1.4) &
(scores["moveSlPips"] == 0.001) &


In [38]:
bestResuts = scores[
    (scores["averageMultiplayer"] == 1.5) &
    (scores["bottomBarrierPips"] == 0.005) &
    (scores["endingHour"] == 12) &
    (scores["openingInterval"] == 1) &
    (scores["slPips"] == 0.0015) &
    (scores["startingHour"] == 8) &
    (scores["tpMultiplier"] == 1.3) &
    (scores["moveSlPips"] == 0.0015)
][['date', 'resultPips']].sort_values('date')

In [39]:
bestResuts.loc[
    bestResuts['resultPips'] > 0
, 'isWin'] = 1
bestResuts = bestResuts.fillna(0)

In [40]:
bestResuts.head(5)

,date,resultPips,isWin
2719,2019-01-10,-0.00009,0.0
6175,2019-01-14,-0.00150,0.0
9631,2019-01-15,-0.00150,0.0
14239,2019-01-17,0.00195,1.0
18847,2019-01-23,0.00073,1.0


#### Merge it with order book data, just to get the data for:
- end of opening interval (q: maybe it is not a good idea to trade?)
- middle hour (q: maybe people on the marked changed the mind?)

In [41]:
base['date'] = base.index.date

In [42]:
labelled = base.reset_index()\
    .merge(bestResuts, 
           right_on = 'date', 
           left_on = 'date', 
           how = 'inner')\

## Opening vs pre-opening period

In [76]:
endOfOpening = labelled[
    (
        (labelled['time'].dt.hour == startingHour - 1) &
        (labelled['time'].dt.minute != 0)
    ) |
    (
        (labelled['time'].dt.hour == startingHour)
    ) |
    (
        (labelled['time'].dt.hour == startingHour + openingInterval) &
        (labelled['time'].dt.minute == 0)
    )
] # so now we have dataset only with hours 7 (pre) and 8 (open)

In [77]:
endOfOpening.loc[
    (
        (endOfOpening['time'].dt.hour == startingHour - 1) &
        (endOfOpening['time'].dt.minute != 0)
    ) |
    (
        (endOfOpening['time'].dt.hour == startingHour) &
        (endOfOpening['time'].dt.minute == 0)
    )
, 'period'] = 'pre_opening'

endOfOpening.loc[
    endOfOpening['period'].isna()
, 'period'] = 'opening'

/home/jacek/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/jacek/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### Difference betwen sum of vol

In [78]:
tmpVol = endOfOpening[['period', 'date', 'vol']]\
    .groupby(['date', 'period'])\
    .sum()\
    .reset_index()\
    .pivot('date', 'period', 'vol')\

In [79]:
tmpVol.head(3)

period,opening,pre_opening
date,,
2019-01-10,504.0,548.0
2019-01-14,618.0,503.0
2019-01-15,458.0,409.0


In [80]:
tmpVol.loc[
    tmpVol['opening'] > tmpVol['pre_opening']
,'isOpeningHigherVolume'] = 1
tmpVol = tmpVol.fillna(0)

In [81]:
tmpVol['percDifference'] = (tmpVol['opening'] - tmpVol['pre_opening']) / tmpVol['pre_opening']

In [82]:
len(bestResuts[bestResuts['isWin'] == 1])/len(bestResuts)

0.52

In [83]:
tmpVol[['percDifference']]\
    .join(bestResuts.set_index('date'))\
    .corr()
    # 'isOpeningHigherVolume'
    #.groupby('isOpeningHigherVolume')\
    #.sum()

,percDifference,resultPips,isWin
percDifference,1.000000,-0.020269,-0.088682
resultPips,-0.020269,1.000000,0.872162
isWin,-0.088682,0.872162,1.000000


## Orderbook data

In [87]:
tmpOrderBook = endOfOpening

In [88]:
# asks if there are more long positions at some market price
for i in range(1, 6):
    tmpOrderBook['hasLvl_'+str(i)+'_UpMoreLongs'] = tmpOrderBook['level_'+str(i)+'_up_l_orders'] > \
                                                tmpOrderBook['level_'+str(i)+'_up_s_orders']
    tmpOrderBook['hasLvl_'+str(i)+'_DownMoreLongs'] = tmpOrderBook['level_'+str(i)+'_down_l_orders'] > \
                                                tmpOrderBook['level_'+str(i)+'_down_s_orders']
    
tmpOrderBook['hasLvl_0_MoreLongs'] = tmpOrderBook['level_0_l_orders'] > \
                                                tmpOrderBook['level_0_s_orders']

/home/jacek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jacek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/jacek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [103]:
tmpOrderBook\
    [['date', 'period', 'hasLvl_0_MoreLongs']]\
    .groupby(['date', 'period'])\
    .sum()\
    .reset_index()\
    .pivot('date', 'period', 'hasLvl_0_MoreLongs')\
    .join(bestResuts.set_index('date'))\
    .head(5)
# first date during pre_opening period had 3 candles, when there were more long orders
# then sell orders, opening period had also 3 candles when there were more long orders.

,opening,pre_opening,resultPips,isWin
date,,,,
2019-01-10,3.0,3.0,-0.00009,0.0
2019-01-14,2.0,0.0,-0.00150,0.0
2019-01-15,0.0,1.0,-0.00150,0.0
2019-01-17,2.0,3.0,0.00195,1.0
2019-01-23,2.0,0.0,0.00073,1.0


#### Not enough data
This mindset (orderbook) is much better for algorithms which will be trading every 20 minutes or hour, so there will be much more candles to make conclusions from